In [ ]:
# Librerías
import tensorflow as tf
import tensorflow_addons as tfa
from vit_keras import vit
from keras.models import load_model
import os
import sys
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import html
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0, Xception, ResNet50, ResNet152, VGG16, DenseNet201, MobileNetV2, InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
import warnings
warnings.filterwarnings('ignore')

<h1 id="Variables">Variables<a class="anchor-link" href="#Variables">¶</a></h1>

In [ ]:
#Elije una configuracion (CF1, CF2, CF3, CF4, CF5) y maquela como True
#######################
CF = 'CF3'
CF1_flag = False
CF2_flag = False
CF3_flag = True
CF4_flag = False
CF5_flag = False
#######################

#Elije la imagen a clasificar con la IC
#######################
name = 'MALIGNO/000011.png'
file_path = './Datos/CF3/imagenes/' + name
#######################

#Especifique los directorios necesarios
#######################
path_or = '.\\Datos\\'
path_img = path_or + f'{CF}\\imagenes\\'
output_path = path_or + f'{CF}\\working\\'
input_path = path_or + f'{CF}\\salidas\\'
#######################

sys.path.append('.\\swintransformer')
from swintransformer import SwinTransformer

# Dimensiones de las imágenes para la representación visual
WIDTH_VIZ = 600
HEIGHT_VIZ = 600


In [ ]:
if CF1_flag or CF4_flag or CF5_flag:
    # Diccionario con las classes CF1
    classes = {0: ('bcc', 'CARCINOMA'),
               1: ('df', 'DERMATOFIBROMA'),
               2: ('bkl', 'LENTIGO'),
               3: ('vasc', 'LESION_VASCULAR'),
               4: ('mel', 'MELANOMA'),
               5: ('nv', 'NEVUS'),
               6: ('akiec', 'QUERATOSIS')}

    cm_plot_classes = ['bcc', 'df', 'bkl', 'vasc', 'mel', 'nv', 'akiec']

if CF2_flag or CF3_flag:
    # Diccionario con las classes CF2
    classes = {0: ('ben', 'BENIGNO'),
               1: ('mal', 'MALIGNO')}

    cm_plot_classes = ['ben', 'mal']

<h1 id="Redes">Redes y Funciones<a class="anchor-link" href="#Redes">¶</a></h1>

In [ ]:
# Función de creación del modelo CNN adhoc
def build_model_cnn_adhoc():
    """
    Crea una red neuronal convolucional con 4 capas convolucionales de 16, 32, 32 y 64 neuronas (kernels) de 3x3 y función de activación ReLU,
    2 capes de MaxPooling 2D de 2x2,
    1 capa Flatten que conecta la salida de la capa convolucional con una entrada de la capa densa,
    2 capas densas de 64 y 32 neuronas completamente conectadas y activación ReLU y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """

    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same')) # capa oculta convolucional con 16 kernels de 3x3 y entrada (28, 28, 3)
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 32 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2)))                                                                      # capa de MaxPooling 2D
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))                            # capa oculta convolucional con 32 kernels de 3x3
    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 64 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))                                                    # capa de MaxPooling 2D
    model.add(Flatten())                                                                                         # capa de Flatten
    model.add(Dense(64, activation='relu'))                                                                      # capa densa completamente conectada de 64 neuronas
    model.add(Dense(32, activation='relu'))                                                                      # capa densa completamente conectada de 32 neuronas
    model.add(Dense(7, activation='softmax'))                                                                    # capa de clasificación con función Softmax para 7 clases
    
    return model

# Función de creación del modelo CNN adhoc binario
def build_model_cnn_adhoc_binary():
    """
    Crea una red neuronal convolucional con 4 capas convolucionales de 16, 32, 32 y 64 neuronas (kernels) de 3x3 y función de activación ReLU,
    2 capes de MaxPooling 2D de 2x2,
    1 capa Flatten que conecta la salida de la capa convolucional con una entrada de la capa densa,
    2 capas densas de 64 y 32 neuronas completamente conectadas y activación ReLU y
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """

    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same')) # capa oculta convolucional con 16 kernels de 3x3 y entrada (28, 28, 3)
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 32 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2)))                                                                      # capa de MaxPooling 2D
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))                            # capa oculta convolucional con 32 kernels de 3x3
    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 64 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))                                                    # capa de MaxPooling 2D
    model.add(Flatten())                                                                                         # capa de Flatten
    model.add(Dense(64, activation='relu'))                                                                      # capa densa completamente conectada de 64 neuronas
    model.add(Dense(32, activation='relu'))                                                                      # capa densa completamente conectada de 32 neuronas
    model.add(Dense(2, activation = 'softmax'))                                                                    # capa de clasificación con función sigmoid para 1 clases
    
    return model



# Función de creación del modelo CNN adhoc Dropout
def build_model_cnn_adhoc_dropout():
    """
    Crea una red neuronal convolucional con 4 capas convolucionales de 16, 32, 32 y 64 neuronas (kernels) de 3x3 y función de activación ReLU,
    2 capes de MaxPooling 2D de 2x2,
    1 capa Flatten que conecta la salida de la capa convolucional con una entrada de la capa densa,
    2 capas densas de 64 y 32 neuronas completamente conectadas y activación ReLU y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """

    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same')) # capa oculta convolucional con 16 kernels de 3x3 y entrada (28, 28, 3)
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 32 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2)))                                                                      # capa de MaxPooling 2D
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))                            # capa oculta convolucional con 32 kernels de 3x3
    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 64 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))                                                    # capa de MaxPooling 2D
    model.add(Flatten())                                                                                         # capa de Flatten
    model.add(Dense(64, activation='relu'))                                                                      # capa densa completamente conectada de 64 neuronas
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))                                                                      # capa densa completamente conectada de 32 neuronas
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))                                                                    # capa de clasificación con función Softmax para 7 clases
    
    return model

In [ ]:
# Función de creación del modelo Visión Transformer ViT-B/16
def build_model_vit():
    """
    Crea un modelo de red neuronal que utiliza la red Vision Transformer con arquitectura ViT-B/16 y
    preentrenada con el dataset imagenet 21K, al que se le sustituyen las capas top de clasificación por las siguientes:
    2 capas de Flatten,
    2 capas de normalización del batch,
    1 capa densa completamente conectadad con activación gausiana y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Importación del modelo ViT · B16 preentrenado y sin la capa de clasificación
    vit_model = vit.vit_b16(image_size = 224,
                            activation = 'softmax',
                            pretrained = True,
                            include_top = False,
                            pretrained_top = False,
                            classes = len(classes))
    
    # Creación del modelo con las nuevas capas para la clasificación
    model = tf.keras.Sequential([vit_model,
                                 keras.layers.Flatten(),                                    # capa de Flatten
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(11, activation = tfa.activations.gelu), # capa densa completamente contectada con activación gausiana
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(7, 'softmax')],                         # capa de clasificación con función Softmax para 7 clases
                                name = 'vit_16')

    return model

# Función de creación del modelo Visión Transformer ViT-B/16 binario

def build_model_vit_binary():
    """
    Crea un modelo de red neuronal que utiliza la red Vision Transformer con arquitectura ViT-B/16 y
    preentrenada con el dataset imagenet 21K, al que se le sustituyen las capas top de clasificación por las siguientes:
    2 capas de Flatten,
    2 capas de normalización del batch,
    1 capa densa completamente conectadad con activación gausiana y
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Importación del modelo ViT · B16 preentrenado y sin la capa de clasificación
    vit_model = vit.vit_b16(image_size = 224,
                            activation = 'softmax',
                            pretrained = True,
                            include_top = False,
                            pretrained_top = False,
                            classes = len(classes))
    
    # Creación del modelo con las nuevas capas para la clasificación
    model = tf.keras.Sequential([vit_model,
                                 keras.layers.Flatten(),                                    # capa de Flatten
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(11, activation = tfa.activations.gelu), # capa densa completamente contectada con activación gausiana
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(2, activation = 'softmax')],              # capa de clasificación con función sigmoid para 1 clases
                                name = 'vit_16')

    return model

# Función de creación del modelo Visión Transformer ViT-B/16 Dropout
def build_model_vit_dropout():
    """
    Crea un modelo de red neuronal que utiliza la red Vision Transformer con arquitectura ViT-B/16 y
    preentrenada con el dataset imagenet 21K, al que se le sustituyen las capas top de clasificación por las siguientes:
    2 capas de Flatten,
    2 capas de normalización del batch,
    1 capa densa completamente conectadad con activación gausiana y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Importación del modelo ViT · B16 preentrenado y sin la capa de clasificación
    vit_model = vit.vit_b16(image_size = 224,
                            activation = 'softmax',
                            pretrained = True,
                            include_top = False,
                            pretrained_top = False,
                            classes = len(classes))
    
    # Creación del modelo con las nuevas capas para la clasificación
    model = tf.keras.Sequential([vit_model,
                                 keras.layers.Flatten(),                                    # capa de Flatten
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 Dropout(0.5),
                                 keras.layers.Dense(11, activation = tfa.activations.gelu), # capa densa completamente contectada con activación gausiana
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 Dropout(0.5),
                                 keras.layers.Dense(7, 'softmax')],                         # capa de clasificación con función Softmax para 7 clases
                                name = 'vit_16')

    return model

In [ ]:
# Función de creación del modelo Swin Transformer
def build_model_SwinT():
    
    """
    Crea un modelo de red transformer que utiliza la red Swin Transformer, preentrenada con el dataset ImageNet-22K,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal transformer
    """
    
    img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode = 'torch'), input_shape = [224, 224, 3])
    pretrained_model = SwinTransformer('swin_large_224',
                                       num_classes = 7,
                                       include_top = False,
                                       pretrained = True,
                                       use_tpu = False)
    
    model = tf.keras.Sequential([
        img_adjust_layer,
        pretrained_model,
        keras.layers.Dense(7, activation = 'softmax')
    ])
 
    return model
# Función de creación del modelo Swin Transformer binary
def build_model_SwinT_binary():
    
    """
    Crea un modelo de red transformer que utiliza la red Swin Transformer, preentrenada con el dataset ImageNet-22K,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal transformer
    """
    
    img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode = 'torch'), input_shape = [224, 224, 3])
    pretrained_model = SwinTransformer('swin_large_224',
                                       num_classes = 7,
                                       include_top = False,
                                       pretrained = True,
                                       use_tpu = False)
    
    model = tf.keras.Sequential([
        img_adjust_layer,
        pretrained_model,
        keras.layers.Dense(2, activation = 'softmax')
    ])
 
    return model

def build_model_SwinT_dropout():
    
    """
    Crea un modelo de red transformer que utiliza la red Swin Transformer, preentrenada con el dataset ImageNet-22K,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal transformer
    """
    
    img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode = 'torch'), input_shape = [224, 224, 3])
    pretrained_model = SwinTransformer('swin_large_224',
                                       num_classes = 7,
                                       include_top = False,
                                       pretrained = True,
                                       use_tpu = False)
    
    model = tf.keras.Sequential([
        img_adjust_layer,
        pretrained_model,
        Dropout(0.5),
        keras.layers.Dense(7, activation = 'softmax')
    ])
 
    return model

In [ ]:
def build_model_EfficientNet():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = EfficientNetB0(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                        # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_EfficientNet_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = EfficientNetB0(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                        # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_EfficientNet_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = EfficientNetB0(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                        # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
def build_model_Xception():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = Xception(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_Xception_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = Xception(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_Xception_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = Xception(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
def build_model_ResNet152():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = ResNet152(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_ResNet152_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = ResNet152(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_ResNet152_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = ResNet152(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
def build_model_VGG16():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = VGG16(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_VGG16_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = VGG16(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_VGG16_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = VGG16(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
def build_model_DenseNet201():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = DenseNet201(include_top = False,                   # no incluir la capa de clasificación
                        input_tensor = inputs,
                        weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_DenseNet201_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = DenseNet201(include_top = False,                   # no incluir la capa de clasificación
                        input_tensor = inputs,
                        weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model


def build_model_DenseNet201_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = DenseNet201(include_top = False,                   # no incluir la capa de clasificación
                        input_tensor = inputs,
                        weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
def build_model_MobileNetV2():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = MobileNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_MobileNetV2_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = MobileNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_MobileNetV2_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = MobileNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
def build_model_InceptionResNetV2():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = InceptionResNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_InceptionResNetV2_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 1 neuronas y función de activación sigmoid.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = InceptionResNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_InceptionResNetV2_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = InceptionResNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
# Función de clasificación de una imagen con un modelo determinado
def ic_class_image(image):

    # Matriz para guardar los resultados de las predicciones de cada modelo para cada clase
    if CF1_flag or CF4_flag or CF5_flag:
        m2 = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64)

    if CF2_flag or CF3_flag:
        m2 = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)

    # Vector para guardar los resultados de la IC
    ic_result = []
    # Vector para guardar la probalidad de cada clase de la IC
    ic_prob = []

    # Vector para guardar el número de redes que han predicho la clase predichca por la IC
    counted_pred = []

    # Predicción con cada modelo
    for key in saved_models.keys():
        model = saved_models[key]
        if key == 1:
            width = WIDTH
            height = HEIGHT
        else:
            width = WIDTH_224
            height = HEIGHT_224

        # Obtención de la matriz de características de la imagen
        img_arr = get_array_from_img(image, width, height)
        # Dimensionamiento del conjunto de datos a las dimensiones esperada por el modelo
        x = np.array(img_arr).reshape(-1, width, height, 3)
        # Normalización
        x = (x - np.mean(x)) / np.std(x)

        # Clasificación de la imagen con el modelo
        pred = model.predict(x, verbose=0)

        # Matriz temporal para guardar los resultados de las predicciones de este modelo
        if CF1_flag or CF4_flag or CF5_flag:
            m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64)

        if CF2_flag or CF3_flag:
            m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)

        # Adición de los resultados a la matriz con los resultados del resto de redes
        for i in range(len(classes)):
            m_temp['C'+str(i)] = [round(pred[0][i],5)]
            
        m2 = m2.append(m_temp)

    # Reinicio de los índices
    m2 = m2.reset_index()
    # Borrado de la columna index
    m2 = m2.drop(['index'], axis = 1)
    # Multiplicación de la matriz de pesos con la matriz de predicciones de la imagen
    m3 = m2.multiply(m1)
    print('Accuracy de cada modelo para cada clase (pesos generales)')
    print(m1)
    print('Accuracy de cada modelo para cada clase (predicciones de la imagen)')
    print(m2)
    print('Accuracy de cada modelo para cada clase (predicciones ponderadas de la imagen)')
    print(m3)

    # Guardado de los resultados de la inteligencia colectiva como la suma de valores de todas las redes
    for i in range(len(classes)):
        ic_result.append(m3['C' + str(i)].sum())

    # Probabilidad de cada clase
    for i in range(len(ic_result)):
        ic_prob.append(ic_result[i]/sum(ic_result))

    # Obtención del id de la clase predicha
    class_id = ic_result.index(max(ic_result))
    # Máximo marcador
    max_score = max(ic_result)

    # Búsqueda de la cantidad de redes que han predicho la clase predicha por la IC
    for i in range(len(saved_models)):
        counted_pred.append(m3.iloc[i].argmax())

    votes_pred = counted_pred.count(class_id)
    # Accuracy de la IC como la máxima puntuación entre las redes que han votado la clase predicha
    ic_accuracy = round(max_score / votes_pred, 2)
    # Código de la lesión cutánea
    lesion_skin_cod = classes[class_id][0]
    # Literal de la lesión cutánea    
    lesion_skin_desc = classes[class_id][1]  

    return class_id, lesion_skin_cod, lesion_skin_desc, ic_accuracy, ic_prob

# Función para la obtención de la matriz de características de una imagen
def get_array_from_img (img_path, width, height):
    """
    Función que lee la imagen intoducida como parámetro de entrada y la reescala a las dimensiones introducidas.
    Devuelve una matriz de dimensiones width x height x 3 (RGB) con las características de la imagen.

    Parámetros de entrada:
    img_path: ruta completa (incluido el nombre del fichero) de la imagen de entrada
    width: ancho de la imagen de salida
    height: alto de la imagen de salida

    Devuelve:
    img_resized: matriz con las características de la imagen de entrada con dimensiones width x height x 3
    """
    try:
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)        # lectura y conversión de formato BGR a RGB
        dim = (width, height)
        img_resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) # reescalado de la imagen
    except Exception as e:
        print(e)
    return img_resized

# Función de clasificación de una imagen con un modelo determinado
def class_img(model_key, image):
    """
    Función que clasifica la imagen introducida con el modelo seleccionado.
    
    Parámetros de entrada:
    model_key: clave del modelo. Diccionario: saved_models.
    image: nombre del fichero que contiene la imagen a clasificar.
    
    Devuelve la clase predicha y el porcentaje de acierto de la clasificación.
    """
    
    # Asignación del modelo
    model = saved_models[model_key]
        
    # Ruta completa de la imagen a clasificar
    img_path = image
        
    if model_key == 1:
        width = WIDTH
        height = HEIGHT
    else:
        width = WIDTH_224
        height = HEIGHT_224
        
    # Obtención de la matriz de características de la imagen
    img_arr = get_array_from_img(img_path, width, height)
    # Dimensionamiento del conjunto de datos a las dimensiones esperada por el modelo
    x = np.array(img_arr).reshape(-1, width, height, 3)
            
    # Normalización
    x = (x - np.mean(x)) / np.std(x)
        
    # Clasificación de la imagen
    result = model.predict(x)
    class_pred = max(result[0])                                        # Clase predicha
    class_ind = list(result[0]).index(class_pred)                      # Índice de la clase predicha
    class_name = classes[class_ind][0] + ' · ' + classes[class_ind][1] # Descriptivo de la clase predicha
    print('***** CLASIFICANDO CON MODELO ' + str(model_key) + '/' + str(len(saved_models)) + ' *****\n')
    print('\tResultado de la clasificación: ' + class_name)
    print('\n\tCerteza del resultado: ' + str(round(class_pred * 100, 3)) + '%\n')
    
    return class_name, round(class_pred * 100, 3)

<h1 id="Ejecucion">Ejecucion<a class="anchor-link" href="#Ejecucion">¶</a></h1>

In [ ]:
# Cargado de la matriz de pesos de los modelos
m1 = pd.read_pickle(input_path + 'm1')
# Matriz de todos los modelos
if CF1_flag or CF2_flag or CF3_flag:
    m1 = m1.drop(6)
    m1 = m1.drop(0)
if CF4_flag:
    m1 = m1.drop(7)
    m1 = m1.drop(6)
    m1 = m1.drop(0)
if CF5_flag:
    m1 = m1.drop(8)
    m1 = m1.drop(7)
    m1 = m1.drop(6)
    m1 = m1.drop(5)
    m1 = m1.drop(0)
m1 = m1.reset_index()
m1 = m1.drop(['index'], axis = 1)
print(m1)

In [ ]:
# Carga de los modelos entrenados
"""if CF1_flag or CF4_flag:
    model1 = build_model_cnn_adhoc()
    model1.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF2_flag or CF3_flag:
    model1 = build_model_cnn_adhoc_binary()
    model1.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF5_flag:
    model1 = build_model_cnn_adhoc_dropout()
    model1.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model1.load_weights(input_path + 'model1.h5')"""

if CF1_flag or CF4_flag:
    model2 = build_model_vit()
    model2.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF2_flag or CF3_flag:
    model2 = build_model_vit_binary()
    model2.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF5_flag:
    model2 = build_model_vit_dropout()
    model2.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model2.load_weights(input_path + 'model2.h5')

if CF1_flag or CF4_flag:
    model3 = build_model_SwinT()
    model3.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF2_flag or CF3_flag:
    model3 = build_model_SwinT_binary()
    model3.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF5_flag:
    model3 = build_model_SwinT_dropout()
    model3.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model3.load_weights(input_path + 'model3.h5')

if CF1_flag or CF4_flag:
    model4 = build_model_EfficientNet()
    model4.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF2_flag or CF3_flag:
    model4 = build_model_EfficientNet_binary()
    model4.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF5_flag:
    model4 = build_model_EfficientNet_dropout()
    model4.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model4.load_weights(input_path + 'model4.h5')

if CF1_flag or CF4_flag:
    model5 = build_model_Xception()
    model5.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF2_flag or CF3_flag:
    model5 = build_model_Xception_binary()
    model5.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF5_flag:
    model5 = build_model_Xception_dropout()
    model5.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model5.load_weights(input_path + 'model5.h5')

if CF1_flag:
    model6 = build_model_ResNet152()
    model6.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model6.load_weights(input_path + 'model6.h5')
if CF2_flag or CF3_flag:
    model6 = build_model_ResNet152_binary()
    model6.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model6.load_weights(input_path + 'model6.h5')

"""if CF1_flag or CF4_flag:
    model7 = build_model_VGG16()
    model7.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF2_flag or CF3_flag:
    model7 = build_model_VGG16_binary()
    model7.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF5_flag:
    model7 = build_model_VGG16_dropout()
    model7.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model7.load_weights(input_path + 'model7.h5')"""

if CF1_flag or CF4_flag:
    model8 = build_model_DenseNet201()
    model8.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model8.load_weights(input_path + 'model8.h5')
if CF2_flag or CF3_flag:
    model8 = build_model_DenseNet201_binary()
    model8.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model8.load_weights(input_path + 'model8.h5')

if CF1_flag or CF4_flag:
    model9 = build_model_MobileNetV2()
    model9.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model9.load_weights(input_path + 'model9.h5')
if CF2_flag or CF3_flag:
    model9 = build_model_MobileNetV2_binary()
    model9.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model9.load_weights(input_path + 'model9.h5')

if CF1_flag or CF4_flag:
    model10 = build_model_InceptionResNetV2()
    model10.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF2_flag or CF3_flag:
    model10 = build_model_InceptionResNetV2_binary()
    model10.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
if CF5_flag:
    model10 = build_model_InceptionResNetV2_dropout()
    model10.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model10.load_weights(input_path + 'model10.h5')

# Diccionario con los ficheros de los mejores modelos guardados
if CF1_flag or CF2_flag or CF3_flag:
    saved_models = {2: model2, 3: model3, 4: model4, 5: model5, 6: model6, 8: model8, 9: model9, 10: model10}
if CF4_flag:
    saved_models = {2: model2, 3: model3, 4: model4, 5: model5, 8: model8, 9: model9, 10: model10}
if CF5_flag:
    saved_models = {2: model2, 3: model3, 4: model4, 5: model5, 10: model10}

<h1 id="Resultados">Resultados<a class="anchor-link" href="#Resultados">¶</a></h1>

In [ ]:
class_id, lesion_skin_cod, lesion_skin_desc, ic_accuracy, ic_prob = ic_class_image(file_path)

In [ ]:


# Gráfico de barras con los resultados de la clasificación
df = pd.DataFrame({'Clases' : cm_plot_classes, 'Prob' : ic_prob})

# Etiquetas para la leyenda
labels = ['bcc · Carcinoma',
          'df · Dermatofibroma',
          'bkl · Lentigo',
          'vasc · Lesiones vasculares',
          'mel · Melanoma',
          'nv · Nevo',
          'akiec · Queratosis']

plt.figure(figsize = (15, 7))
ax = sns.barplot(data = df, x = 'Clases', y = 'Prob', hue = 'Clases', dodge = False)
ax.set_xlabel('Tipo de lesión', size = 15)
ax.set_ylabel('% de probabilidad de la lesión', size = 15)
ax.set_title('Resultados de la clasificación de la imagen', size = 20)

h, l = ax.get_legend_handles_labels()
ax.legend(h, labels, title = 'Leyenda', bbox_to_anchor = (1.02, 1), loc = 'upper left', borderaxespad = 0)
plt.savefig(output_path + 'resultado.jpg', bbox_inches = 'tight')
plt.tight_layout()
plt.show()

print('Tipo de lesión: ' + str(lesion_skin_cod) + ' · ' + str(lesion_skin_desc)) 
print('Acierto: ' + str(round(ic_accuracy*100,2)))


In [ ]:
metadata = pd.read_csv(path_or + f'metadata_{CF}.csv', delimiter=';')

In [ ]:
new_image = file_path
# Columnas
columns = ['class_name', 'accuracy', 'accuracy_total']
# Dataframe para almacenar los resultados de todos los modelos
results = pd.DataFrame(columns = columns)

# Iteración por todos los modelos
for key in saved_models.keys():
    # Llamada a la función de clasificación
    class_name, accuracy = class_img(key, new_image)
    res = [[class_name, accuracy, round(accuracy / (len(saved_models)),3)]]
    results = results.append(pd.DataFrame(res, columns = columns), ignore_index = True)

# Consenso entre todos los modelos
print('... INICIANDO CONSENSO ...\n')
print('***** CONSENSO ENTRE ' + str(len(saved_models)) + ' MODELOS *****')
results_sum = results.groupby('class_name', as_index = False)['accuracy_total'].sum().sort_values(by = ['accuracy_total'], ascending = False)
class_name_res = results_sum['class_name'].head(1).to_string(index = False)
accuracy_total_res = results_sum['accuracy_total'].head(1).to_string(index = False)
consensus = len(results[results['class_name'] == class_name_res])
por_consensus = consensus/len(saved_models)

# El consenso tiene que ser la mitad más 1 de los modelos utilizados
if por_consensus < 0.5:
    print('\nNo hay consenso en la clasificación de la imagen')
    print('\nResultado de la clasificación: ' + str(class_name_res))
    print('Consenso: ' + str(consensus) + ' de ' + str(len(saved_models)) + ' · ' + str(round(por_consensus*100,3)) + '%') 
    print('Certeza del resultado: ' + accuracy_total_res + '%\n')
else:
    print('\nResultado de la clasificación: ' + str(class_name_res))
    print('Consenso: ' + str(consensus) + ' de ' + str(len(saved_models)) + ' · ' + str(round(por_consensus*100,3)) + '%') 
    print('Certeza del resultado: ' + accuracy_total_res + '%\n')

# Visualización de la imagen clasificada
plt.axis('off')
plt.imshow(cv2.resize(get_array_from_img(new_image, WIDTH_VIZ, HEIGHT_VIZ),
                      (WIDTH_VIZ, HEIGHT_VIZ),
                      interpolation = cv2.INTER_CUBIC))


clase_real = classes[int(metadata.loc[metadata['image'] == name]['label'])][1]

print('***** CLASE REAL: {} *****'.format(clase_real))